In [ ]:
import pandas as pd
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt
from gudhi.representations.kernel_methods import PersistenceFisherDistance, \
                                                SlicedWassersteinDistance
from tqdm import tqdm
import tensorflow as tf

In [ ]:
def Cubical(X, dim, card):
    # Parameters: X (image),
    #             dim (homological dimension), 
    #             card (number of persistence diagram points, sorted by distance-to-diagonal)

    # Compute the persistence pairs with Gudhi
    cc = gd.CubicalComplex(dimensions=X.shape, top_dimensional_cells=tf.reshape(X, (-1,)))
    cc.persistence()
    try:
        cof = cc.cofaces_of_persistence_pairs()[0][dim]
    except IndexError:
        cof = np.array([])
        
    Xs = X.shape

    if len(cof) > 0:
        # Sort points with distance-to-diagonal
        pers = [X[np.unravel_index(cof[idx,1], Xs)] - X[np.unravel_index(cof[idx,0], Xs)] for idx in range(len(cof))]
        perm = np.argsort(pers)
        cof = cof[perm[::-1]]
    
    # Retrieve and ouput image indices/pixels corresponding to positive and negative simplices
    D = len(Xs)
    ocof = np.array([0 for _ in range(D*card*2)])
    count = 0
    for idx in range(0,min(2*card, 2*cof.shape[0]),2):
        ocof[D*idx:D*(idx+1)]     = np.unravel_index(cof[count,0], Xs)
        ocof[D*(idx+1):D*(idx+2)] = np.unravel_index(cof[count,1], Xs)
        count += 1
    return list(np.array(ocof, dtype=np.int32))

class CubicalModel(tf.keras.Model):
    def __init__(self, X, dim=1, card=50):
        super(CubicalModel, self).__init__()
        self.X = X
        self.dim = dim
        self.card = card
        
    def call(self):
        d, c, D = self.dim, self.card, len(self.X.shape)
        XX = tf.reshape(self.X, [1, self.X.shape[0], self.X.shape[1]])
        
        # Turn numpy function into tensorflow function
        CbTF = lambda X: tf.py_function(Cubical, [X, d, c], [tf.int32 for _ in range(2*D*c)])
        
        # Compute pixels associated to positive and negative simplices 
        # Don't compute gradient for this operation
        inds = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(CbTF, XX, dtype=[tf.int32 for _ in range(2*D*c)]))
        
        # Get persistence diagram by simply picking the corresponding entries in the image
        dgm = tf.reshape(tf.gather_nd(self.X, tf.reshape(inds, [-1,D])), [-1,2])
        return dgm
    
def get_SWD(d1, d2, num_directions = 10):
    thetas = tf.linspace(-np.pi/2, np.pi/2, num_directions)[np.newaxis,:-1]
    lines = tf.concat([tf.cos(thetas), tf.sin(thetas)], axis = 0)
    aprox1 = tf.matmul(d1, lines)
    aprox1_diag = tf.matmul(tf.broadcast_to(tf.reduce_sum(d1, -1, keepdims = True)/2, (len(d1),2)), lines)
    aprox2 = tf.matmul(d2, lines)
    aprox2_diag = tf.matmul(tf.broadcast_to(tf.reduce_sum(d2, -1, keepdims = True)/2, (len(d2),2)), lines)
    A = tf.sort(tf.concat([aprox1, aprox2_diag], axis = 0), axis = 0)
    B = tf.sort(tf.concat([aprox2, aprox1_diag], axis = 0), axis = 0)
    L1 = tf.reduce_sum(tf.abs(A - B), axis = 0)
    return tf.reduce_mean(L1)

def get_diractions(img, C, max_val = 10e3):
    inds = np.argwhere(img > 0)
    IX, IY = max_val * np.ones(img.shape), max_val * np.ones(img.shape)
    for k in range(len(inds)):
        IX[inds[k,0], inds[k,1]] = inds[k,0]
        IY[inds[k,0], inds[k,1]] = inds[k,1]
                
    II = tf.math.cos(C)*IX + tf.math.sin(C)*IY
    II = II/max_val
    dgm = CubicalModel(II, dim=hdim, card=hcard).call()
    return dgm

def get_all_dist(diags, num_directions = 10):
    
    size = diags.shape[0]
    all_dist = []
    for i in range(size):
        for j in range(i + 1, size):
            all_dist.append(get_SWD(diags[i], diags[j], num_directions))
            
    return tf.stack(all_dist, axis=0, name='stack')

In [ ]:
I = np.array(pd.read_csv('datasets/mnist_train.csv', header = None))

In [ ]:
def get_y(y):
    v = np.zeros(10)
    v[y] = 1
    return tf.reshape(tf.convert_to_tensor(v, tf.float32), (1,-1))

In [ ]:
#без обучения фильтров
num_filters = 10

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
diractions = tf.Variable(trainable = False, name = 'diractions', initial_value = tf.random.uniform((num_filters,1), -np.pi/2, np.pi/2))
print(diractions)
num_epochs = 10
hdim = 0
hcard = 50
bce = tf.keras.losses.BinaryCrossentropy()

out_size = int(num_filters*(num_filters - 1)/2)
W1 = tf.Variable(initial_value = np.random.normal(size = (out_size, 32)), trainable=True, dtype = tf.float32, name = 'w1')
W2 = tf.Variable(initial_value = np.random.normal(size = (32, 10)), trainable=True, dtype = tf.float32, name = 'w2')

for epoch in range(num_epochs+1):
    for ind, img in enumerate(I):
        y = get_y(img[0])
        image = np.reshape(img[1:], [28,28])
        image = image / 255
        image = np.array(image > 0, np.float64)
        diags = []
        
        with tf.GradientTape() as tape:
            FDiags = lambda X : tf.py_function(get_diractions, [image, X], tf.float32)
            diags =  tf.map_fn(FDiags, diractions, dtype=tf.float32)
            dist_tensor = tf.reshape(get_all_dist(diags), (1, -1))
            
            l1 = tf.nn.relu(dist_tensor@W1)
            
            l2 = tf.nn.softmax(l1@W2)
            
            loss = bce(y, l2)
            loss = tf.reshape(loss, (1,))

            gradients = tape.gradient(loss, [W1, W2])
            optimizer.apply_gradients(zip(gradients, [W1, W2]))
            
            if(ind%100 == 0):
                print(loss)
                print(diractions)
                

<tf.Variable 'diractions:0' shape=(10, 1) dtype=float32, numpy=
array([[ 1.5446924 ],
       [ 0.8005568 ],
       [ 1.0657297 ],
       [-0.97108966],
       [-1.1747761 ],
       [-0.6289613 ],
       [ 0.18318713],
       [ 0.00731444],
       [ 1.1650361 ],
       [ 1.1399466 ]], dtype=float32)>
tf.Tensor([0.323674], shape=(1,), dtype=float32)
<tf.Variable 'diractions:0' shape=(10, 1) dtype=float32, numpy=
array([[ 1.5446924 ],
       [ 0.8005568 ],
       [ 1.0657297 ],
       [-0.97108966],
       [-1.1747761 ],
       [-0.6289613 ],
       [ 0.18318713],
       [ 0.00731444],
       [ 1.1650361 ],
       [ 1.1399466 ]], dtype=float32)>
tf.Tensor([0.32496375], shape=(1,), dtype=float32)
<tf.Variable 'diractions:0' shape=(10, 1) dtype=float32, numpy=
array([[ 1.5446924 ],
       [ 0.8005568 ],
       [ 1.0657297 ],
       [-0.97108966],
       [-1.1747761 ],
       [-0.6289613 ],
       [ 0.18318713],
       [ 0.00731444],
       [ 1.1650361 ],
       [ 1.1399466 ]], dtype=float32

       [ 1.1399466 ]], dtype=float32)>
tf.Tensor([0.3141554], shape=(1,), dtype=float32)
<tf.Variable 'diractions:0' shape=(10, 1) dtype=float32, numpy=
array([[ 1.5446924 ],
       [ 0.8005568 ],
       [ 1.0657297 ],
       [-0.97108966],
       [-1.1747761 ],
       [-0.6289613 ],
       [ 0.18318713],
       [ 0.00731444],
       [ 1.1650361 ],
       [ 1.1399466 ]], dtype=float32)>
tf.Tensor([0.4551157], shape=(1,), dtype=float32)
<tf.Variable 'diractions:0' shape=(10, 1) dtype=float32, numpy=
array([[ 1.5446924 ],
       [ 0.8005568 ],
       [ 1.0657297 ],
       [-0.97108966],
       [-1.1747761 ],
       [-0.6289613 ],
       [ 0.18318713],
       [ 0.00731444],
       [ 1.1650361 ],
       [ 1.1399466 ]], dtype=float32)>
tf.Tensor([0.17314602], shape=(1,), dtype=float32)
<tf.Variable 'diractions:0' shape=(10, 1) dtype=float32, numpy=
array([[ 1.5446924 ],
       [ 0.8005568 ],
       [ 1.0657297 ],
       [-0.97108966],
       [-1.1747761 ],
       [-0.6289613 ],
       [ 0.1

In [ ]:
#с обучением фильтров
num_filters = 10

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
diractions = tf.Variable(trainable = True, name = 'diractions', initial_value = tf.random.uniform((num_filters,1), -np.pi/2, np.pi/2))
print(diractions)
num_epochs = 10
hdim = 0
hcard = 50
bce = tf.keras.losses.BinaryCrossentropy()

out_size = int(num_filters*(num_filters - 1)/2)
W1 = tf.Variable(initial_value = np.random.normal(size = (out_size, 32)), trainable=True, dtype = tf.float32, name = 'w1')
W2 = tf.Variable(initial_value = np.random.normal(size = (32, 10)), trainable=True, dtype = tf.float32, name = 'w2')

for epoch in range(num_epochs+1):
    for ind, img in enumerate(I):
        y = get_y(img[0])
        image = np.reshape(img[1:], [28,28])
        image = image / 255
        image = np.array(image > 0, np.float64)
        diags = []
        
        with tf.GradientTape() as tape:
            FDiags = lambda X : tf.py_function(get_diractions, [image, X], tf.float32)
            diags =  tf.map_fn(FDiags, diractions, dtype=tf.float32)
            dist_tensor = tf.reshape(get_all_dist(diags), (1, -1))
            
            l1 = tf.nn.relu(dist_tensor@W1)
            
            l2 = tf.nn.softmax(l1@W2)
            
            loss = bce(y, l2)
            loss = tf.reshape(loss, (1,))

            gradients = tape.gradient(loss, [W1, W2])
            optimizer.apply_gradients(zip(gradients, [W1, W2]))
            
            if(ind%100 == 0):
                print(loss)
                print(diractions)

In [ ]:
diractions

[<tf.Variable 'dir0:0' shape=() dtype=float32, numpy=-0.88494223>,
 <tf.Variable 'dir1:0' shape=() dtype=float32, numpy=0.29739916>,
 <tf.Variable 'dir2:0' shape=() dtype=float32, numpy=0.75910497>,
 <tf.Variable 'dir3:0' shape=() dtype=float32, numpy=0.14101169>,
 <tf.Variable 'dir4:0' shape=() dtype=float32, numpy=1.5218873>]

In [ ]:
thetas = tf.linspace(-np.pi/2, np.pi/2, num_directions)[np.newaxis,:-1]

In [ ]:
lines = tf.concat([tf.cos(thetas), tf.sin(thetas)], axis = 0)

In [ ]:
tf.reduce_sum(thetas, -1)

<tf.Tensor: id=33, shape=(1,), dtype=float32, numpy=array([-1.5707961], dtype=float32)>

In [ ]:
a = np.array([[1.1,2.2], [2,3]])
daig = tf.constant(a, np.float32)

In [ ]:
tf.matmul(tf.broadcast_to(tf.reduce_sum(daig, -1, keepdims = True)/2, (len(daig),2)), lines)

<tf.Tensor: id=80, shape=(2, 9), dtype=float32, numpy=
array([[-1.6500002 , -0.9861596 , -0.20337391,  0.6039419 ,  1.3384135 ,
         1.9114525 ,  2.253942  ,  2.324573  ,  2.1148262 ],
       [-2.5       , -1.4941812 , -0.30814242,  0.9150634 ,  2.027899  ,
         2.89614   ,  3.4150634 ,  3.52208   ,  3.2042818 ]],
      dtype=float32)>

In [ ]:
tf.broadcast_to(tf.reduce_sum(daig, -1, keepdims = True)/2, (len(daig),2))

<tf.Tensor: id=66, shape=(2, 2), dtype=float64, numpy=
array([[1.65, 1.65],
       [2.5 , 2.5 ]])>

In [ ]:
daig

<tf.Tensor: id=44, shape=(2, 2), dtype=float64, numpy=
array([[1.1, 2.2],
       [2. , 3. ]])>